In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier

In [4]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalaanced

ImportError: cannot import name 'classification_report_imbalaanced' from 'imblearn.metrics' (C:\Users\austi\anaconda3\lib\site-packages\imblearn\metrics\__init__.py)

# Read the CSV and Perform Basic Data Cleaning

In [ ]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [6]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]



NameError: name 'columns' is not defined

In [ ]:
# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)



# Split the Data into Training and Testing

In [ ]:
# Create our features
X = df.copy()
X.drop("loan_status", axis=1, inplace=True)
X = pd.get_dummies(X)
X.head()

In [16]:
# Create our target
y = df["loan_status"]
y[:5]

0    low_risk
1    low_risk
2    low_risk
3    low_risk
4    low_risk
Name: loan_status, dtype: object

In [7]:
X.describe()

NameError: name 'X' is not defined

In [18]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [19]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [20]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [21]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

In [22]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [26]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=78)
brf.fit(X_train, y_train)
y_pred = brf.predict(X_test)
y_pred

array(['low_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object)

In [27]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.7848905230958412

In [30]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix

brfc_matrix = confusion_matrix(y_test, y_pred)
brfc_matrix

array([[   61,    31],
       [ 1596, 15517]], dtype=int64)

In [32]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred)) 

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.66      0.91      0.07      0.78      0.59        92
   low_risk       1.00      0.91      0.66      0.95      0.78      0.62     17113

avg / total       0.99      0.91      0.66      0.95      0.78      0.62     17205



In [36]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE

importances = brf.feature_importances_
importances_sorted = sorted(zip(brf.feature_importances_, X.columns), reverse=True)
importances_sorted

[(0.07392208280891524, 'last_pymnt_amnt'),
 (0.07330719736528421, 'total_rec_prncp'),
 (0.07237501703183195, 'total_pymnt_inv'),
 (0.05903599879032377, 'total_pymnt'),
 (0.04824912220818261, 'total_rec_int'),
 (0.027945081701714863, 'int_rate'),
 (0.022805069792995628, 'issue_d_Jan-2019'),
 (0.01786171353915809, 'revol_bal'),
 (0.017261367702290906, 'mo_sin_old_rev_tl_op'),
 (0.017015203539438525, 'installment'),
 (0.016740694309834422, 'dti'),
 (0.016353685148731315, 'bc_util'),
 (0.01609670087939147, 'out_prncp'),
 (0.015698164190808536, 'issue_d_Mar-2019'),
 (0.01475861144463269, 'max_bal_bc'),
 (0.014647165184499468, 'out_prncp_inv'),
 (0.014317870979462022, 'mths_since_recent_inq'),
 (0.014170175626345628, 'bc_open_to_buy'),
 (0.013347928482753839, 'annual_inc'),
 (0.013202663973241721, 'mths_since_rcnt_il'),
 (0.012967740614174699, 'total_bal_ex_mort'),
 (0.012877532593957165, 'total_il_high_credit_limit'),
 (0.01245371489238309, 'total_rev_hi_lim'),
 (0.012019263719051203, 'tota

### Easy Ensemble Classifier

In [38]:
# Train the EasyEnsembleClassifier
# YOUR CODE HERE
from imblearn.ensemble import EasyEnsembleClassifier
eec= EasyEnsembleClassifier(n_estimators= 100, random_state= 68)

eec.fit(X_train_scaled, y_train)


EasyEnsembleClassifier(n_estimators=100, random_state=68)

In [39]:


# Calculated the balanced accuracy score
# YOUR CODE HERE
eec_predict = eec.predict(X_test_scaled)

In [41]:
# Display the confusion matrix
# YOUR CODE HERE
eec_score = balanced_accuracy_score(y_test, eec_predict)
print(eec_score)

0.9319208763233646


In [43]:
# Print the imbalanced classification report
# YOUR CODE HERE
eec_class = classification_report_imbalanced(y_test, eec_predict)
print(eec_class)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.92      0.94      0.14      0.93      0.87        92
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17113

avg / total       0.99      0.94      0.92      0.96      0.93      0.87     17205

